# Overview
- Library Installation
- Dataset - histograms of wet tank air pressures from buses
- Anomaly Detection with COSMO method
- Reviewing anomaly scores with repair events

# Installing required libraries

Let's start by installing required libraries.

In [ ]:
!pip install -U -q PyDrive

In [ ]:
import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import os, math

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Dataset

We start by loading the dataset.

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_dataset = "1cNE-d8-43XY6s7eIOSHZSq1P2gbn2d34"
#https://drive.google.com/file/d/1cNE-d8-43XY6s7eIOSHZSq1P2gbn2d34/view?usp=sharing
downloaded = drive.CreateFile({'id': file_dataset})
downloaded.GetContentFile('lab1_example_dataset.csv')
print(os.listdir())
df_hist=pd.read_csv('lab1_example_dataset.csv')

In [ ]:
df_hist.head(5)

In [ ]:
df_hist.describe()

In [ ]:
df_hist["t"] = pd.to_datetime(df_hist["t"])

# Anomaly Detection with COSMO method

Implementing the COSMO method according to the pseudocode, and material in slides.



In [ ]:
# Initialization

random_state = 42
nn_samples_reference_group = 50
histogram_param = [xx for xx in df_hist.columns if "bin" in xx]

df_hist['date_week'] = df_hist['t'].dt.to_period('W')
week_lst = list(df_hist['date_week'].unique())
T_u = 4 # 4 weeks

df_hist["z_score"] = np.nan
df_hist["anomaly_score"] = np.nan

# Selected metric - Hellinger distance
def hellinger_dist(u, v):
    a = 0.0
    for v1,v2 in zip(np.array(u).flat,np.array(v).flat):
        a += (math.sqrt(v1)-math.sqrt(v2))**2
    return math.sqrt(a/2)

# Computing distance matrix
def compute_distance_matrix(samples):
    def symmetrize(a):
        return a + a.T - np.diag(a.diagonal())
    
    dist_matrix = np.zeros((len(samples), len(samples)))
    
    for ir in range(len(samples)):
        for ic in range(ir, len(samples)):
            dist_matrix[ir, ic] = hellinger_dist(samples[ir], samples[ic])

    return symmetrize(dist_matrix)

# Acquire the most central pattern
def get_mcp(samples):
    dist_matrix = compute_distance_matrix(samples)
    mcp_idx = dist_matrix.sum(axis=1).argmin()
    return samples[mcp_idx], dist_matrix[mcp_idx]


In [ ]:
for iweek, week in enumerate(week_lst):

    # acquire the reference group

    # compute pairwise distance matrix

    # acquire most central pattern

    # compute z-score

In [ ]:
def normcdf(x, mu, sigma):
    t = x-mu
    y = 0.5*math.erfc(-t/(sigma*math.sqrt(2.0)))
    if y>1.0:
        y = 1.0
    return y

def getArithmeticP_val(gMu, n):
    if n == 0:
        return float('NaN')
    amu = 0.5 # mean
    asigma = np.sqrt(1.0/(12.0*n)) # Standard deviation for the mean
    return normcdf(gMu, amu, asigma)

# for each unit, compute the anomaly score based on the z-scores
for i_vid, vid in enumerate(df_hist["id"].unique()):
    
    veh_readings = df_hist[df_hist["id"]==vid].copy().sort_values(by=['t'])
    
    for iweek, week in enumerate(week_lst):

        # compute mean values of the z-score over the period of T_u

        # compute arithmatic p-value

# Anomaly scores and repair events

Let's review anomaly scores and a few repair events.

In [ ]:
events = {
    0: ["2012-07-02", 1],
    4: ["2012-06-04", 0],
    6: ["2012-09-17", 1],
    12: ["2012-03-23", 0],
    13: ["2012-10-18", 1]
}

plt.clf()

for i_vid, vid in enumerate(df_hist["id"].unique()):
    
    veh_readings = df_hist[df_hist["id"]==vid].copy().sort_values(by=['t'])
    
    plt.figure(figsize=(13, 2))
    plt.plot(veh_readings["t"], veh_readings["anomaly_score"])
    plt.ylabel(vid)
    plt.xlim(df_hist["t"].min(), df_hist["t"].max())
    plt.ylim([0, 15])
    
    print(vid, events)
    if vid in events:
        if events[vid][1]:
            plt.axvline(x=pd.to_datetime(events[vid][0]), c="r")
        else:
            plt.axvline(x=pd.to_datetime(events[vid][0]), c="y")

In [ ]:
plt.clf()

for i_vid, vid in enumerate(df_hist["id"].unique()):

    veh_readings = df_hist[df_hist["id"]==vid].copy().sort_values(by=['t'])
    
    plt.figure(figsize=(13, 2))
    plt.scatter(veh_readings["t"], veh_readings["z_score"], s=5, marker="o")
    plt.xlim(df_hist["t"].min(), df_hist["t"].max())
    plt.ylabel(vid)

# Reference

- Fan, Y., Nowaczyk, S., & Rögnvaldsson, T. (2015). Evaluation of self-organized approach for predicting compressor faults in a city bus fleet. Procedia Computer Science, 53, 447-456.

- Fan, Yuantao, Sławomir Nowaczyk, Thorsteinn Rögnvaldsson, and Eric Aislan Antonelo. "Predicting air compressor failures with echo state networks." PHM Society European Conference. Vol. 3. No. 1. 2016.
